# The Classifiers before Feature Engineering

In [1]:
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
# from sklearn.svm import LinearSVC, SVC
# from sklearn.tree import ExtraTreeClassifier, DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier
# from mlxtend.classifier import StackingClassifier
# from sklearn.cross_validation import cross_val_score

In [30]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import ExtraTreeClassifier, DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.cross_validation import cross_val_score
import pandas as pd
from sklearn.utils import class_weight
import numpy as np

In [3]:
base_dir_tr='training/'
base_dir_vl='validation/'
weight_log='weight_logs/'

In [16]:
dl_tr=pd.read_csv(base_dir_tr+'length_string_2.csv')
dl_vl=pd.read_csv(base_dir_vl+'length_string_2.csv')         

Xtr=dl_tr[['len_title_des','len_short_des','price_rupiah_st']].as_matrix()
Xvl=dl_vl[['len_title_des','len_short_des','price_rupiah_st']].as_matrix()

#### conciseness_train
xtr=open(base_dir_tr+'conciseness_train.labels')
concise=[]
for l in xtr:
    concise.append(int(l))
    
#### clarity_train
xtr=open(base_dir_tr+'clarity_train.labels')
clarity=[]
for l in xtr:
    clarity.append(int(l))

dl_tr['conci']=concise
dl_tr['clari']=clarity

yco=dl_tr['conci']
ycl=dl_tr['clari']

In [18]:
# X_train, X_test, yco_train, yco_test = train_test_split(Xtr, yco, test_size=0.3, random_state=21)

In [31]:
clas_weight = class_weight.compute_class_weight('balanced', np.unique(ycl), ycl)
clas_weight

array([ 8.82798054,  0.53001928])

In [42]:
randomforest = RandomForestClassifier(random_state = 21, n_estimators= 100)
boosting3 = GradientBoostingClassifier( random_state = 21, n_estimators = 200)

In [43]:
randomforest_cl = RandomForestClassifier(random_state = 21, n_estimators= 100,class_weight=dict(zip([1,0],clas_weight)))
boosting3_cl = GradientBoostingClassifier( random_state = 21, n_estimators = 200)

In [44]:
classifiers = [randomforest, boosting3]

In [45]:
print('3-fold cross validation:\n')

for clf, label in zip([randomforest, boosting3], 
                      ['random_forest','Gradient_boosting']):

    scores = cross_val_score(clf, Xtr, yco, cv=3, scoring='accuracy',verbose=1)
    print("Accuracy: %0.4f (+/- %0.4f) [%s]" 
          % (scores.mean(), scores.std(), label))

3-fold cross validation:



[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   12.2s finished


Accuracy: 0.6971 (+/- 0.0056) [random_forest]
Accuracy: 0.7336 (+/- 0.0056) [Gradient_boosting]


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    6.7s finished


In [46]:
print('3-fold cross validation:\n')

for clf, label in zip([randomforest_cl, boosting3_cl], 
                      ['random_forest','Gradient_boosting']):

    scores = cross_val_score(clf, Xtr, ycl, cv=3, scoring='accuracy',verbose=1)
    print("Accuracy: %0.4f (+/- %0.4f) [%s]" 
          % (scores.mean(), scores.std(), label))

3-fold cross validation:



[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   10.7s finished


Accuracy: 0.9327 (+/- 0.0004) [random_forest]
Accuracy: 0.9428 (+/- 0.0002) [Gradient_boosting]


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    6.6s finished


# Feature Engineering

## GridSearch Optim

# Gradient Boosting

In [47]:
from sklearn.grid_search import RandomizedSearchCV

c:\users\aska\anaconda2\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [48]:
param_Boosting = {'learning_rate' : [ 1, 0.33, 0.1, 0.033, 0.01, 0.0033],
           'n_estimators': [100, 200, 300, 400, 500, 700,
                           900, 1000, 1200, 1500, 2000, 3000, 4000, 5000]}

In [55]:
boostOptim = RandomizedSearchCV(GradientBoostingClassifier(), param_Boosting,n_iter = 30 , cv = 3,verbose=1)

In [ ]:
boostOptim.fit(Xtr, yco)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


In [ ]:
boostOptim.best_score_

In [ ]:
boostOptim.best_params_